In [208]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import using libraries

In [209]:
import numpy as np 
import pandas as pd 
import torch
import random
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import plotly.express as px
from torch.utils.data import DataLoader

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

datasets

In [210]:
train_data = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test_data = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

In [211]:
train_data.head()

In [212]:
SL_labels = {}
for l in set(train_data.label):
    SL_labels[l] = list(train_data.label).count(l)
SL_labels

По данной диаграмме можем сделать вывод, что данный датасет содержит 24 буквы, по 1000 изображений каждой

In [213]:
px.pie(values=SL_labels.values(), names=SL_labels.keys())

# Подготовка датасета

In [214]:
y_train = train_data['label'] 
y_test = test_data['label']
del train_data['label'] 
del test_data['label']

In [215]:
dtr_labels = y_train.unique()
dtr_labels = np.sort(dtr_labels)
dtr_labels

In [216]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

y_train[:3]

Функция для преобразования в изобажение

In [238]:
def preproc_img(x):
    x = x/255
    x = x.reshape(-1, 28, 28, 1)
    return(x)

In [239]:
x_train = preproc_img(train_data.values) 
x_test =  preproc_img(test_data.values)

In [221]:
def show_images(images,labels):
    fig,ax = plt.subplots(2,5)
    fig.set_size_inches(10, 6)
    k =0
    for i in range(2):
        for j in range(5):
            ax[i,j].imshow(images[k] , cmap='gray')
            ax[i,j].set_title(str(dtr_labels[np.argmax(y_train[k])]))
            k = k+1;
    plt.tight_layout()

In [222]:
show_images(x_train, y_train)

In [223]:
show_images(x_test, y_test)

In [224]:
x_train.shape

# Постройка модели

In [257]:
class ExNet(torch.nn.Module):
    def __init__(self):
        super(ExNet, self).__init__()
        self.Conv1 = nn.Sequential(nn.Conv2d(1, 32, 5),
                                   #nn.MaxPool2d(2),
                                      nn.ReLU(),
                                      nn.BatchNorm2d(32))
        self.Conv2 = nn.Sequential(nn.Conv2d(32, 64, 3), 
                                      nn.BatchNorm2d(64),)
                                    #nn.MaxPool2d(2))
        self.Conv3 = nn.Sequential(nn.Conv2d(64, 128, 3),
                                      nn.MaxPool2d(2),
                                      nn.ReLU(),
                                      nn.BatchNorm2d(128))
        self.Conv4 = nn.Sequential(nn.Conv2d(128, 256, 3), 
                                      nn.MaxPool2d(2),
                                      nn.ReLU(),            
                                      nn.BatchNorm2d(256))
        self.Linear1 = nn.Linear(4096, 256)
        self.dropout=nn.Dropout(0.1)
        self.Linear3 = nn.Linear(256, 25)
        
    def forward(self, x):
        x = self.Conv1(x)
        x = self.Conv2(x)
        x = self.Conv3(x)
        x = self.Conv4(x)
        x=self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.Linear1(x)
        x = self.dropout(x)
        x = self.Linear3(x)
        return x                        
"""
a = np.random.rand(28,28)
#print(a)
data = torch.tensor(a).float()
print(data.shape)
# data.unsqueeze_(0).unsqueeze_(0)
data= data.expand(16, 1 ,-1,-1)
print(data.shape)

n=ExNet()
print("Start")
o = n(data)
print(o)
"""
#model = ExNet()

In [253]:
num_epochs = 5 
num_classes = 25 
batch_size = 100 
learning_rate = 0.001

In [258]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [259]:
x_train = x_train.permute(0, 3, 1, 2)
x_test = x_test.permute(0, 3, 1, 2)

In [260]:
x_train.shape

**функция потерь и оптимизатор**

In [261]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

**тренировка сети**

In [262]:
#total_step = len(train)
def train(net, x_train, y_train, X_test, y_test):
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    batch_size = 100

    loss_list = []
    acc_list = []

    for epoch in range(10):
        order = np.random.permutation(len(x_train))
        for start_index in range(0, len(x_train), batch_size):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            x_batch = x_train[batch_indexes]
            y_batch = y_train[batch_indexes]

            preds = net.forward(x_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            x_batch

        net.eval()
        test_preds = net.forward(x_test)
        test_loss_history.append(loss(test_preds, y_test).data)

        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data
        test_accuracy_history.append(accuracy)

        print(accuracy)
    del net
    return loss_list, acc_list

accuracies = {}
losses = {}

In [263]:
accuracies['cifar_net'], losses['cifar_net'] = \
    train(ExNet(), x_train, y_train, x_test, y_test)